In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import requests

from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from io import BytesIO
from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =
        
        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(src,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                pic = Image(BytesIO(resp.content))

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

总数量：507
[状态：ok，尝试次数：0] - 
[剩余数量：477] - [当前时间：15:35:03]

[状态：ok，尝试次数：0] - 
[剩余数量：476] - [当前时间：15:35:03]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-1035SX-1-KIT.jpg
[剩余数量：475] - [当前时间：15:35:06]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-1041S-1-KIT.jpg
[剩余数量：474] - [当前时间：15:35:06]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-0460SA-1-KIT.jpg
[剩余数量：473] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-0470SX-1-KIT.jpg
[剩余数量：472] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-0490SX-1-KIT.jpg
[剩余数量：471] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-0420S-1-KIT.jpg
[剩余数量：470] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-1038S-1-KIT.jpg
[剩余数量：469] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-1047S-1-KIT.jpg
[剩余数量：468] - [当前时间：15:35:07]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/583/3-0719SHWP-1-KIT.jpg
[剩余数量：467] - [当前时间：15:35:07]